# This is test jupyter notebook file, it tests the "tools/get_flood_sampling_data.py".

## Note: Some of the functions in the "tools/get_flood_sampling_data.py" file operate under specific assumptions, and the expected output is unpredictable even under these assumptions. Therefore, they were not tested in this test, but they all working properly in the project jupyter notebook files(try to re-run all the jupyter notebooks under from source code folder)

In [1]:
!earthengine authenticate

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=cccmTDscGLFaMjtgQNVm-BfFGByLC4G-HFvRJAd1oVU&tc=OG-zhrfoX6hSq8qFiPBatxA_KaZ9O9xFgSwpBO5j1nc&cc=X3nes0ooY-S2GlHya4pB6LlpN3DEFwk6zzolLCVRK0Y

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1Adeu5BUeBycVF0tqVy1vUbAVk5o8juTvbguV8kMsmARclnIc5-s2VEu2gXs

Successfully saved authorization token.


In [2]:
# Prepare a bit of packages:
from google.colab import drive
drive.mount('/content/gdrive')
import sys
tool_folder_dir = "/content/gdrive/MyDrive/irp_project_111/source code"
sys.path.append(tool_folder_dir)
!pip install ipytest

Mounted at /content/gdrive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.8 MB/s eta 0:00:00


In [3]:
"""
Author: Chaofan Wu
Student ID: 02285924
Email: cw522@ic.ac.uk
Project Name: Predicting flood risk in Ghana
Supervisors:
    Sesinam Dagadu(MEng)
    Yves Plancherel(PhD)
Company: SnooCODE
Date: 08/2023
"""

import ipytest
ipytest.autoconfig()
from tools import get_flood_sampling_data
import ee
import pandas as pd
from io import StringIO


def test_initialize_ee_and_get_flood_data():
    """Tests the initialize_ee_and_get_flood_data function."""
    # Initialize the Earth Engine library
    ee.Initialize()

    flood_data = get_flood_sampling_data.initialize_ee_and_get_flood_data()
    assert isinstance(flood_data, ee.ImageCollection), "The returned object is not an ImageCollection."


def test_calculate_flooded_area_proportion():
    """Tests the calculate_flooded_area_proportion function."""
    # Initialize the Earth Engine library
    ee.Initialize()

    # Assuming you have a specific flood event and ROI for testing
    event_id = 2320
    global_flood_data = get_flood_sampling_data.initialize_ee_and_get_flood_data()
    flood_event = global_flood_data.filter(ee.Filter.eq('id', event_id)).first()
    ROI = ee.Geometry.Rectangle([-0.87, 8.05, -0.37, 8.55])
    proportion = get_flood_sampling_data.calculate_flooded_area_proportion(flood_event, ROI, 250)

    # Depending on the actual flood data and ROI you have, you might know the expected proportion.
    expected_proportion = 0.059500888385087755
    assert abs(proportion - expected_proportion) < 1e-7, f"Expected {expected_proportion} but got {proportion}."


def test_coordinates_dict_to_df():
    """Test the coordinates_dict_to_df function."""
    # Sample input
    coordinates_dict = {'event1': [(1.0, 1.0), (2.0, 2.0)], 'event2': [(3.0, 3.0, 1), (4.0, 4.0, 0)]}
    flooded = 1
    prefix = 'test'

    # Expected output
    expected_df = pd.DataFrame([
        {'Flood_ID': 'test_event1', 'Lon': 1.0, 'Lat': 1.0, 'Flooded': 1},
        {'Flood_ID': 'test_event1', 'Lon': 2.0, 'Lat': 2.0, 'Flooded': 1},
        {'Flood_ID': 'test_event2', 'Lon': 3.0, 'Lat': 3.0, 'Flooded': 1},
        {'Flood_ID': 'test_event2', 'Lon': 4.0, 'Lat': 4.0, 'Flooded': 0},
    ])

    # Function output
    output_df = get_flood_sampling_data.coordinates_dict_to_df(coordinates_dict, flooded, prefix)

    # Asserting equality
    pd.testing.assert_frame_equal(output_df, expected_df)


def test_add_max_duration():
    """Test the add_max_duration function."""
    # Sample input
    input_df = pd.DataFrame({
        'Flood_ID': ['event1', 'event1', 'event2', 'event2', 'event3'],
        'Duration': [2, 4, 1, 3, 5]
    })

    # Function output
    output_df = get_flood_sampling_data.add_max_duration(input_df)

    # Expected output
    expected_df = pd.DataFrame({
        'Flood_ID': ['event1', 'event1', 'event2', 'event2', 'event3'],
        'Duration': [2, 4, 1, 3, 5],
        'Max_Duration': [4, 4, 3, 3, 5]
    })

    # Asserting equality
    pd.testing.assert_frame_equal(output_df, expected_df)


def test_add_event_start_and_end_date():
    """Test the add_event_start_and_end_date function."""
    # Sample input
    input_df = pd.DataFrame({
        'Flood_ID': ['event1_1', 'event2_2'],
        'Max_Duration': [5, 10]
    })

    # Create a simulated historical flood information CSV file contents
    historical_info_csv = """id,Start_Date
                1,2021-01-01
                2,2021-02-01
                """

    # Use StringIO to read CSV content as a DataFrame
    historical_info_df = pd.read_csv(StringIO(historical_info_csv))

    # Save modelled historical flood information DataFrame to CSV file
    historical_info_path = "/tmp/historical_flood_info.csv"
    historical_info_df.to_csv(historical_info_path, index=False)

    # Call add_event_start_and_end_date function
    output_df = get_flood_sampling_data.add_event_start_and_end_date(input_df, historical_info_path)

    # Create a DataFrame with the expected output
    expected_df = pd.DataFrame({
        'Flood_ID': ['event1_1', 'event2_2'],
        'Max_Duration': [5, 10],
        'Event_Start_Date': pd.to_datetime(['2021-01-01', '2021-02-01']),
        'Event_End_Date': pd.to_datetime(['2021-01-06', '2021-02-11'])
    })

    # Use the assert_frame_equal method of pandas to assert whether two DataFrames are equal.
    pd.testing.assert_frame_equal(output_df, expected_df)


#Start test:
ipytest.run()

...

Processing <Max_Duration> column:   0%|          | 0/3 [00:00<?, ?it/s]

.

Processing <Event_Start_Date> & <Event_End_Date> columns:   0%|          | 0/2 [00:00<?, ?it/s]

.                                                                                        [100%]
5 passed in 6.73s


<ExitCode.OK: 0>